In [1]:
train_path = 'train_hand_fin.txt'
dev_path = 'dev_hand_fin.txt'
test_path = 'test_hand_fin.txt'
train_path_2 = 'skweak_labelled_20_all.txt'
model_checkpoint = "xlm-roberta-base"

batch_size = 16
learning_rate = 5e-5
epoch = 4
weight_decay = 0.1

save_folder_name = 'xlm_alldata'
save_model_name = 'xlm_alldata_fin'

In [2]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 10.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 13.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
import re
import pandas as pd
from datasets import Dataset

Load dataset

In [4]:
def load_data_weak(path):
  file = open(path, 'r')
  lines = file.readlines()

  tokens = []
  ner_tags = []

  temp_toks = []
  temp_tags = []

  for i in lines:
    if i == ' \t \n':
      tokens.append(temp_toks)
      ner_tags.append(temp_tags)

      temp_toks = []
      temp_tags = []

    else:
      temp = re.split('\t',i)
      temp_toks.append(temp[0])
      temp_tags.append(re.sub('\n','',temp[-1]))
  
  data = pd.DataFrame({'tokens':tokens,
                       'ner_tags':ner_tags})
  return data


In [5]:
def load_data(path):
  file = open(path, 'r')
  lines = file.readlines()

  tokens = []
  ner_tags = []

  temp_toks = []
  temp_tags = []

  for i in lines:
    if i == '\n':
      tokens.append(temp_toks)
      ner_tags.append(temp_tags)

      temp_toks = []
      temp_tags = []

    else:
      temp = re.split('\t',i)
      temp_toks.append(temp[0])
      temp_tags.append(re.sub('\n','',temp[-1]))
  
  data = pd.DataFrame({'tokens':tokens,
                       'ner_tags':ner_tags})
  return data


In [15]:
train1 = load_data(train_path)
train2 = load_data_weak(train_path_2)
traindata = pd.concat([train1,train2])
traindata = traindata.reset_index(drop = True)
test = load_data(test_path)
dev = load_data(dev_path)

train2.head()

,tokens,ner_tags
0,"[Maßnahmenbekanntgabe, zu, MA, 37, ,, Prüfung,...","[O, O, B-ORG, I-ORG, O, O, O, O]"
1,[INHALTSVERZEICHNIS],[O]
2,[ABKÜRZUNGSVERZEICHNIS],[O]
3,"[bzw., beziehungsweise, Nr., Nummer]","[O, O, O, O]"
4,"[Erledigung, des, Prüfungsberichtes, Der, Stad...","[O, O, O, O, B-ORG, I-ORG, O, O, O, O, O, O, B..."


In [7]:
label2id = {'O': 0,'B-ORG':1,'I-ORG':2}
id2label = {0 :'O', 1:'B-ORG', 2:'I-ORG'}
label_list = ['O','B-ORG','I-ORG']

In [8]:
from transformers import (
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer
)

In [9]:
config = AutoConfig.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label = id2label,
    label2id = label2id
)


Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    config=config
)

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-st

In [11]:
def align_labels(data, label_encoding_dict):
    tokenized_inputs = tokenizer(data["tokens"],
                        max_length = 128, padding = 'max_length',
                        truncation=True, is_split_into_words=True)

    label_id_temp = {}
    for i, label in enumerate(data['ner_tags']):
        label_id_temp.update({i:label})

    word_ids = tokenized_inputs.word_ids(batch_index=0)

    labels = []
    for w_id in word_ids:
        if w_id == None:
            labels.append(-100)
        else:
            tag = label_id_temp[w_id]
            labels.append(label2id[tag])

    tokenized_inputs["labels"] = labels

    return tokenized_inputs

def tokenized_for_bert(df):
  for_bert = df.copy()
  for_bert['input_ids'] = ""
  # for_bert['token_type_ids'] = ""
  for_bert['attention_mask']= ""
  for_bert['labels'] = ""

  for index, row in df.iterrows():
      inputs = align_labels(row,label2id)
      for_bert.at[index,'input_ids'] = inputs['input_ids']
      # for_bert.at[index,'token_type_ids'] = inputs['token_type_ids']
      for_bert.at[index,'attention_mask'] = inputs['attention_mask']
      for_bert.at[index,'labels'] = inputs['labels']
  for_bert = for_bert.drop('tokens', axis = 1)
  for_bert = for_bert.drop('ner_tags', axis = 1)
  dataset_for_bert = Dataset.from_pandas(for_bert)
  print(dataset_for_bert)
  return dataset_for_bert

In [16]:
train_dataset = tokenized_for_bert(traindata)
dev_dataset = tokenized_for_bert(dev)
test_dataset = tokenized_for_bert(test)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 13856
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1227
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1226
})


In [17]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached seqeval-1.2.2.tar.gz (43 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=6e17f02a6d0a5bec936aa81ae35164592261dfbc7db4dfaceec7e31f1e9ba597
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval


In [19]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch
from seqeval.metrics import classification_report

In [20]:
import numpy as np
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    
    result = classification_report(true_labels, true_predictions, output_dict = True)

    print(result)
    
    return result['ORG']

In [21]:

args = TrainingArguments(
    save_folder_name,
    save_strategy='epoch',
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay = weight_decay,
    metric_for_best_model = 'f1-score',
    save_total_limit=5,
    load_best_model_at_end = True
)

data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print("Training...")
trainer.train()
print("Evaluating...")
trainer.evaluate()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 13856
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3464
  Number of trainable parameters = 277455363
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Training...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1-score,Support
1,0.052700,0.101638,0.640906,0.805696,0.713915,2563
2,0.030100,0.071810,0.673357,0.811549,0.736023,2563
3,0.022200,0.070411,0.719038,0.792821,0.754129,2563
4,0.015600,0.073490,0.725740,0.822864,0.771256,2563


***** Running Evaluation *****
  Num examples = 1227
  Batch size = 16


{'ORG': {'precision': 0.6409062693978895, 'recall': 0.8056964494732735, 'f1-score': 0.7139152981849611, 'support': 2563}, 'micro avg': {'precision': 0.6409062693978895, 'recall': 0.8056964494732735, 'f1-score': 0.7139152981849611, 'support': 2563}, 'macro avg': {'precision': 0.6409062693978895, 'recall': 0.8056964494732735, 'f1-score': 0.7139152981849611, 'support': 2563}, 'weighted avg': {'precision': 0.6409062693978895, 'recall': 0.8056964494732735, 'f1-score': 0.7139152981849611, 'support': 2563}}


...

{'ORG': {'precision': 0.725739848589126, 'recall': 0.8228638314475224, 'f1-score': 0.7712561711464618, 'support': 2563}, 'micro avg': {'precision': 0.725739848589126, 'recall': 0.8228638314475224, 'f1-score': 0.7712561711464618, 'support': 2563}, 'macro avg': {'precision': 0.725739848589126, 'recall': 0.8228638314475224, 'f1-score': 0.7712561711464618, 'support': 2563}, 'weighted avg': {'precision': 0.725739848589126, 'recall': 0.8228638314475224, 'f1-score': 0.7712561711464618, 'support': 2563}}


Saving model checkpoint to xlm_alldata/checkpoint-3464
Configuration saved in xlm_alldata/checkpoint-3464/config.json
Model weights saved in xlm_alldata/checkpoint-3464/pytorch_model.bin
tokenizer config file saved in xlm_alldata/checkpoint-3464/tokenizer_config.json
Special tokens file saved in xlm_alldata/checkpoint-3464/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from xlm_alldata/checkpoint-3464 (score: 0.7712561711464618).
***** Running Evaluation *****
  Num examples = 1227
  Batch size = 16


Evaluating...


{'ORG': {'precision': 0.725739848589126, 'recall': 0.8228638314475224, 'f1-score': 0.7712561711464618, 'support': 2563}, 'micro avg': {'precision': 0.725739848589126, 'recall': 0.8228638314475224, 'f1-score': 0.7712561711464618, 'support': 2563}, 'macro avg': {'precision': 0.725739848589126, 'recall': 0.8228638314475224, 'f1-score': 0.7712561711464618, 'support': 2563}, 'weighted avg': {'precision': 0.725739848589126, 'recall': 0.8228638314475224, 'f1-score': 0.7712561711464618, 'support': 2563}}


{'eval_loss': 0.07348997890949249,
 'eval_precision': 0.725739848589126,
 'eval_recall': 0.8228638314475224,
 'eval_f1-score': 0.7712561711464618,
 'eval_support': 2563,
 'eval_runtime': 9.8587,
 'eval_samples_per_second': 124.459,
 'eval_steps_per_second': 7.81,
 'epoch': 4.0}

In [22]:
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 1226
  Batch size = 16


{'ORG': {'precision': 0.761437908496732, 'recall': 0.8583389149363697, 'f1-score': 0.8069899244332494, 'support': 2986}, 'micro avg': {'precision': 0.761437908496732, 'recall': 0.8583389149363697, 'f1-score': 0.8069899244332494, 'support': 2986}, 'macro avg': {'precision': 0.761437908496732, 'recall': 0.8583389149363697, 'f1-score': 0.8069899244332494, 'support': 2986}, 'weighted avg': {'precision': 0.761437908496732, 'recall': 0.8583389149363697, 'f1-score': 0.8069899244332494, 'support': 2986}}


PredictionOutput(predictions=array([[[ 0.16568334, -0.29245967, -0.65356016],
        [ 8.003172  , -3.9120426 , -4.063191  ],
        [ 7.9998617 , -3.6915455 , -4.347385  ],
        ...,
        [ 0.26726088, -0.26674482, -0.65746903],
        [ 0.26726088, -0.26674482, -0.65746903],
        [ 0.26726088, -0.26674482, -0.65746903]],

       [[ 0.24372938, -0.27918297, -0.75030625],
        [ 7.312022  , -2.9872077 , -4.343879  ],
        [ 0.9675486 ,  4.1245656 , -4.0847225 ],
        ...,
        [ 0.31884155, -0.26110741, -0.7400975 ],
        [ 0.31884155, -0.26110741, -0.7400975 ],
        [ 0.31884155, -0.26110741, -0.7400975 ]],

       [[ 1.8386996 , -0.8982274 , -1.4485468 ],
        [ 8.319336  , -4.093825  , -4.1363664 ],
        [ 8.219573  , -4.0045547 , -4.1795163 ],
        ...,
        [ 8.205224  , -3.7749767 , -4.275988  ],
        [ 7.7103977 , -3.3799992 , -4.4372654 ],
        [ 2.0151966 , -0.87968236, -1.4311126 ]],

       ...,

       [[ 0.35966012,  0.066114